To download Qwen2.5-32B-Instruct in local, run:

pip install huggingface_hub

huggingface-cli download Qwen/Qwen2.5-32B-Instruct --local-dir "./Qwen2.5-32B"

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
# !cp "/content/drive/MyDrive/Projet nlp/sanad_dataset.zip" /content/sanad_dataset.zip

In [3]:
# !ls -lh /content/

In [1]:
!pip install datasets
print("✔️ datasets est bien installé !")

✔️ datasets est bien installé !


In [2]:
import torch
print("GPU disponible :", torch.cuda.is_available())
print("Nom du GPU :", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "Aucun GPU détecté")
print("Nombre de GPU disponibles :", torch.cuda.device_count())
for i in range(torch.cuda.device_count()):
    print(f"GPU {i} : {torch.cuda.get_device_name(i)}")

GPU disponible : True
Nom du GPU : Tesla T4
Nombre de GPU disponibles : 2
GPU 0 : Tesla T4
GPU 1 : Tesla T4


In [3]:
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM
import os
import pandas as pd
import numpy as np
from datasets import Dataset

In [17]:
# Vérifier si CUDA est disponible
DEVICE = "cuda" if torch.cuda.is_available() else -1
print(f"🔹 Utilisation de l'appareil : {DEVICE}")

🔹 Utilisation de l'appareil : cuda


In [21]:
# Charger le modèle et le tokenizer
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

tokenizer = AutoTokenizer.from_pretrained("facebook/mbart-large-50")
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/mbart-large-50").to(device)

if torch.cuda.device_count() > 1:
    print(f"🛠️ Mode Multi-GPU activé sur {torch.cuda.device_count()} GPU")
    model = torch.nn.DataParallel(model)

# Définir le pipeline de résumé
# Définir le pipeline de résumé
if torch.cuda.device_count() > 1:
    summarization_pipeline = pipeline("summarization", model=model.module, tokenizer=tokenizer, device=0)
else:
    summarization_pipeline = pipeline("summarization", model=model, tokenizer=tokenizer, device=0)

Device set to use cuda:0


🛠️ Mode Multi-GPU activé sur 2 GPU


In [5]:
df_clean = pd.read_csv("/kaggle/input/data-arabic/sanad_dataset_clean.csv")

In [8]:
df_clean.head()

,filename,category,text
0,4705.txt,Culture,يبدو أن أزمة الحفل الموسيقي، الذي تدخلت نقابة ...
1,2497.txt,Culture,كشف المغني البريطاني من أصل لبناني ميكا عن مشر...
2,1124.txt,Culture,أكد الفنان الكويتي عبدالحسين عبدالرضا أنه يعتب...
3,0182.txt,Culture,دشنت مدينة نابلس، أمس الاثنين، أكبر جدارية تضم...
4,0681.txt,Culture,"أقرت المغنية البريطانية ""اديل"" بأنها بكت طويلا..."


In [9]:
print(df_clean.shape)
print(f"🔹 Taille du DataFrame complet : {df_clean.shape}")

# Diviser le DataFrame en 10 chunks
chunks = np.array_split(df_clean, 2000)
print(f"🔹 Taille du premier chunk : {chunks[0].shape}")

# Travailler uniquement avec le premier chunk
first_chunk = chunks[0]

# Convertir le premier chunk en Dataset Hugging Face
dataset = Dataset.from_pandas(first_chunk)

(205760, 3)
🔹 Taille du DataFrame complet : (205760, 3)
🔹 Taille du premier chunk : (103, 3)


In [10]:
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

In [10]:
df=dataset.to_pandas()
df.head()

,filename,category,text
0,4705.txt,Culture,يبدو أن أزمة الحفل الموسيقي، الذي تدخلت نقابة ...
1,2497.txt,Culture,كشف المغني البريطاني من أصل لبناني ميكا عن مشر...
2,1124.txt,Culture,أكد الفنان الكويتي عبدالحسين عبدالرضا أنه يعتب...
3,0182.txt,Culture,دشنت مدينة نابلس، أمس الاثنين، أكبر جدارية تضم...
4,0681.txt,Culture,"أقرت المغنية البريطانية ""اديل"" بأنها بكت طويلا..."


In [ ]:
from torch.utils.data import DataLoader
from tqdm import tqdm  # Importer tqdm pour afficher la progression

batch_size = 16  # Taille du batch pour utiliser les 2 GPU

# Créer un DataLoader pour envoyer les batchs aux GPU
dataloader = DataLoader(dataset["text"], batch_size=batch_size, shuffle=False)

# Fonction pour générer des résumés en batch avec affichage de l'avancement
def generate_summaries(dataloader):
    summaries = []
    total_batches = len(dataloader)  # Nombre total de batchs
    
    print(f"🔹 Génération des résumés pour {total_batches} batchs...")

    for batch in tqdm(dataloader, total=total_batches, desc="🔄 Résumé en cours"):
        batch_texts = [text[:1024] for text in batch]

        # Exécuter le résumé sur GPU
        batch_summaries = summarization_pipeline(batch_texts, max_length=100, min_length=20, do_sample=False)

        summaries.extend([s["summary_text"] for s in batch_summaries])
    
    return summaries

# Générer les résumés en batch avec affichage de la progression
df["summary"] = generate_summaries(dataloader)

print("✔️ Génération des résumés terminée !")


🔹 Génération des résumés pour 7 batchs...


🔄 Résumé en cours:  43%|████▎     | 3/7 [01:34<02:05, 31.44s/it]

In [ ]:
# Sauvegarder les résumés
df.to_csv("/kaggle/working/sanad_dataset_summarized.csv", index=False)
print("✔️ Résumés sauvegardés dans : /kaggle/working/sanad_dataset_summarized.csv")